# ARC Prize 2025 - Model Training

Train neural network models for ARC pattern recognition.

**Author:** Andrew Jewell Sr.  
**Company:** AutomataNexus, LLC  
**Date:** September 26, 2024

## Models:
1. **MINERVA** - Strategic Pattern Analysis & Decision Making
2. **ATLAS** - Spatial Transformations & Structural Support  
3. **IRIS** - Color Pattern Recognition & Harmony
4. **CHRONOS** - Temporal Sequences & Evolution Detection
5. **PROMETHEUS** - Creative Pattern Generation & Foresight

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install torch torchvision matplotlib numpy pandas tqdm onnx onnxruntime plotly scikit-learn

# For Colab GPU info
!nvidia-smi

In [ ]:
# Clone repository from GitHub
!git clone https://github.com/AutomataControls/Arc2025.git
%cd Arc2025

# Or if you need to upload files manually:
# from google.colab import files
# uploaded = files.upload()  # Upload arc_models.py and other files

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from typing import Dict, List, Tuple
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from datetime import datetime

# Import our models
from models.arc_models import (
    MinervaNet, AtlasNet, IrisNet, ChronosNet, PrometheusNet,
    create_models
)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

## 2. Dataset Preparation

In [ ]:
# Download ARC dataset
!wget https://github.com/fchollet/ARC-AGI/raw/main/data/training.tar.gz
!tar -xzf training.tar.gz
!wget https://github.com/fchollet/ARC-AGI/raw/main/data/evaluation.tar.gz
!tar -xzf evaluation.tar.gz

In [ ]:
class ARCDataset(Dataset):
    """ARC dataset with pattern labeling for training"""
    
    def __init__(self, data_path: str, max_grid_size: int = 30):
        self.max_grid_size = max_grid_size
        self.tasks = []
        self.pattern_labels = {
            'rotation': 0, 'reflection': 1, 'scaling': 2, 'translation': 3,
            'color_mapping': 4, 'symmetry': 5, 'object_movement': 6,
            'counting': 7, 'logical': 8, 'composite': 9
        }
        
        # Load all JSON files
        for filename in os.listdir(data_path):
            if filename.endswith('.json'):
                with open(os.path.join(data_path, filename), 'r') as f:
                    task = json.load(f)
                    task['filename'] = filename
                    self.tasks.append(task)
        
        # Create training pairs with pseudo-labels
        self.pairs = []
        for task in self.tasks:
            for example in task.get('train', []):
                self.pairs.append({
                    'input': np.array(example['input']),
                    'output': np.array(example['output']),
                    'task_id': task['filename'],
                    'pattern_label': self._detect_pattern_type(example)
                })
        
        print(f"Loaded {len(self.tasks)} tasks with {len(self.pairs)} training pairs")
        
        # Pattern distribution
        pattern_counts = {}
        for pair in self.pairs:
            label = pair['pattern_label']
            pattern_name = list(self.pattern_labels.keys())[label]
            pattern_counts[pattern_name] = pattern_counts.get(pattern_name, 0) + 1
        print("\nPattern distribution:")
        for pattern, count in sorted(pattern_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"  {pattern}: {count} ({count/len(self.pairs)*100:.1f}%)")
    
    def _detect_pattern_type(self, example: Dict) -> int:
        """Simple heuristic to assign pattern labels"""
        input_grid = np.array(example['input'])
        output_grid = np.array(example['output'])
        
        # Check for size changes (scaling)
        if input_grid.shape != output_grid.shape:
            if output_grid.size > input_grid.size:
                return self.pattern_labels['scaling']
            else:
                return self.pattern_labels['counting']
        
        # Check for rotations
        for k in [1, 2, 3]:
            if np.array_equal(np.rot90(input_grid, k), output_grid):
                return self.pattern_labels['rotation']
        
        # Check for reflections
        if np.array_equal(np.flip(input_grid, axis=0), output_grid):
            return self.pattern_labels['reflection']
        if np.array_equal(np.flip(input_grid, axis=1), output_grid):
            return self.pattern_labels['reflection']
        
        # Check for color changes
        if set(input_grid.flatten()) != set(output_grid.flatten()):
            return self.pattern_labels['color_mapping']
        
        # Check for symmetry
        if (np.array_equal(output_grid, np.flip(output_grid, axis=0)) or 
            np.array_equal(output_grid, np.flip(output_grid, axis=1))):
            return self.pattern_labels['symmetry']
        
        # Default to composite
        return self.pattern_labels['composite']
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        pair = self.pairs[idx]
        
        # Convert to one-hot encoding
        input_grid = self._grid_to_tensor(pair['input'])
        output_grid = self._grid_to_tensor(pair['output'])
        pattern_label = torch.tensor(pair['pattern_label'], dtype=torch.long)
        
        return input_grid, output_grid, pattern_label
    
    def _grid_to_tensor(self, grid: np.ndarray) -> torch.Tensor:
        """Convert grid to one-hot tensor and pad to max size"""
        h, w = grid.shape
        
        # One-hot encode
        one_hot = np.zeros((10, self.max_grid_size, self.max_grid_size))
        for i in range(min(h, self.max_grid_size)):
            for j in range(min(w, self.max_grid_size)):
                color = grid[i, j]
                one_hot[color, i, j] = 1
        
        return torch.FloatTensor(one_hot)

# Create dataset
full_dataset = ARCDataset('training')

# Split into train/val
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)

print(f"\nTrain size: {len(train_dataset)}, Val size: {len(val_dataset)}")

## 3. Training Functions with Metrics

In [ ]:
class ModelTrainer:
    """Comprehensive model trainer with metrics tracking"""
    
    def __init__(self, model, model_name, device):
        self.model = model.to(device)
        self.model_name = model_name
        self.device = device
        
        # Training history
        self.history = {
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'learning_rates': [],
            'pattern_accuracies': {}
        }
        
        # Best model tracking
        self.best_val_acc = 0
        self.best_epoch = 0
        
    def train_epoch(self, loader, optimizer, criterion):
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for input_grid, output_grid, pattern_label in tqdm(loader, desc='Training'):
            input_grid = input_grid.to(self.device)
            output_grid = output_grid.to(self.device)
            pattern_label = pattern_label.to(self.device)
            
            optimizer.zero_grad()
            
            # Forward pass (model-specific)
            if self.model_name in ['minerva', 'iris']:
                outputs = self.model(input_grid, output_grid)
                logits = outputs['pattern_logits'] if 'pattern_logits' in outputs else outputs['pattern_type_logits']
            elif self.model_name == 'atlas':
                outputs = self.model(input_grid)
                # Use transform params as proxy for pattern classification
                transform_params = outputs['transform_params']
                logits = transform_params[:, :10]  # First 10 dims for classification
            elif self.model_name == 'chronos':
                outputs = self.model([input_grid])
                logits = outputs['evolution_type_logits']
            elif self.model_name == 'prometheus':
                outputs = self.model(input_grid)
                logits = outputs['synthesis_strategy_logits'][:, :10]
            
            loss = criterion(logits, pattern_label)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = torch.max(logits.data, 1)
            total += pattern_label.size(0)
            correct += (predicted == pattern_label).sum().item()
        
        return total_loss / len(loader), correct / total
    
    def validate(self, loader, criterion):
        self.model.eval()
        total_loss = 0
        correct = 0
        total = 0
        
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for input_grid, output_grid, pattern_label in loader:
                input_grid = input_grid.to(self.device)
                output_grid = output_grid.to(self.device)
                pattern_label = pattern_label.to(self.device)
                
                # Forward pass (same as training)
                if self.model_name in ['minerva', 'iris']:
                    outputs = self.model(input_grid, output_grid)
                    logits = outputs['pattern_logits'] if 'pattern_logits' in outputs else outputs['pattern_type_logits']
                elif self.model_name == 'atlas':
                    outputs = self.model(input_grid)
                    transform_params = outputs['transform_params']
                    logits = transform_params[:, :10]
                elif self.model_name == 'chronos':
                    outputs = self.model([input_grid])
                    logits = outputs['evolution_type_logits']
                elif self.model_name == 'prometheus':
                    outputs = self.model(input_grid)
                    logits = outputs['synthesis_strategy_logits'][:, :10]
                
                loss = criterion(logits, pattern_label)
                total_loss += loss.item()
                
                _, predicted = torch.max(logits.data, 1)
                total += pattern_label.size(0)
                correct += (predicted == pattern_label).sum().item()
                
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(pattern_label.cpu().numpy())
        
        return total_loss / len(loader), correct / total, all_preds, all_labels
    
    def train(self, train_loader, val_loader, epochs=50, lr=1e-3):
        optimizer = optim.AdamW(self.model.parameters(), lr=lr, weight_decay=0.01)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
        criterion = nn.CrossEntropyLoss()
        
        print(f"\nTraining {self.model_name.upper()} for {epochs} epochs...")
        print(f"Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        
        for epoch in range(epochs):
            # Train
            train_loss, train_acc = self.train_epoch(train_loader, optimizer, criterion)
            
            # Validate
            val_loss, val_acc, val_preds, val_labels = self.validate(val_loader, criterion)
            
            # Update history
            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['learning_rates'].append(scheduler.get_last_lr()[0])
            
            # Save best model
            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                self.best_epoch = epoch
                torch.save(self.model.state_dict(), f'{self.model_name}_best.pt')
            
            # Print progress
            print(f'Epoch [{epoch+1}/{epochs}] '
                  f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} '
                  f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
            
            scheduler.step()
        
        # Final validation with best model
        self.model.load_state_dict(torch.load(f'{self.model_name}_best.pt'))
        val_loss, val_acc, val_preds, val_labels = self.validate(val_loader, criterion)
        
        print(f"\nBest validation accuracy: {self.best_val_acc:.4f} at epoch {self.best_epoch+1}")
        
        return val_preds, val_labels

## 4. Train All Models

In [ ]:
# Create models
models = create_models()

# Training results storage
training_results = {}

# Train each model
for model_name, model in models.items():
    print(f"\n{'='*60}")
    print(f"Training {model_name.upper()} - {model.description}")
    print(f"{'='*60}")
    
    trainer = ModelTrainer(model, model_name, device)
    
    # Train with appropriate epochs
    epochs = 30 if model_name in ['minerva', 'prometheus'] else 25
    val_preds, val_labels = trainer.train(train_loader, val_loader, epochs=epochs)
    
    # Store results
    training_results[model_name] = {
        'trainer': trainer,
        'history': trainer.history,
        'best_acc': trainer.best_val_acc,
        'val_preds': val_preds,
        'val_labels': val_labels
    }

## 5. Generate Visualizations with Plotly

In [ ]:
def create_training_plots(model_name, history):
    """Create training visualizations using Plotly"""
    
    # Training history plot
    fig = go.Figure()
    
    # Add traces
    fig.add_trace(go.Scatter(
        x=list(range(1, len(history['train_loss']) + 1)),
        y=history['train_loss'],
        name='Train Loss',
        line=dict(color='blue', width=2)
    ))
    
    fig.add_trace(go.Scatter(
        x=list(range(1, len(history['val_loss']) + 1)),
        y=history['val_loss'],
        name='Val Loss',
        line=dict(color='red', width=2)
    ))
    
    # Create second y-axis for accuracy
    fig.add_trace(go.Scatter(
        x=list(range(1, len(history['train_acc']) + 1)),
        y=history['train_acc'],
        name='Train Acc',
        line=dict(color='green', width=2, dash='dash'),
        yaxis='y2'
    ))
    
    fig.add_trace(go.Scatter(
        x=list(range(1, len(history['val_acc']) + 1)),
        y=history['val_acc'],
        name='Val Acc',
        line=dict(color='orange', width=2, dash='dash'),
        yaxis='y2'
    ))
    
    # Update layout
    fig.update_layout(
        title=f'{model_name.upper()} Training History',
        xaxis_title='Epoch',
        yaxis=dict(title='Loss', side='left'),
        yaxis2=dict(title='Accuracy', side='right', overlaying='y'),
        hovermode='x unified',
        width=800,
        height=500
    )
    
    return fig

def create_confusion_matrix_plot(model_name, y_true, y_pred, class_names):
    """Create confusion matrix using Plotly"""
    
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=cm,
        x=class_names,
        y=class_names,
        colorscale='Blues',
        text=cm,
        texttemplate='%{text}',
        textfont={"size": 12}
    ))
    
    fig.update_layout(
        title=f'{model_name.upper()} Confusion Matrix',
        xaxis_title='Predicted',
        yaxis_title='True',
        width=600,
        height=600
    )
    
    return fig

def create_performance_comparison():
    """Create model performance comparison chart"""
    
    model_names = []
    accuracies = []
    
    for model_name, results in training_results.items():
        model_names.append(model_name.upper())
        accuracies.append(results['best_acc'] * 100)
    
    fig = go.Figure(data=[
        go.Bar(
            x=model_names,
            y=accuracies,
            text=[f'{acc:.1f}%' for acc in accuracies],
            textposition='auto',
            marker_color=['#6b46c1', '#2c5aa0', '#ff6b6b', '#4ecdc4', '#f7b731']
        )
    ])
    
    fig.update_layout(
        title='Model Performance Comparison',
        xaxis_title='Model',
        yaxis_title='Validation Accuracy (%)',
        yaxis_range=[0, 100],
        width=800,
        height=500
    )
    
    return fig

# Generate plots for each model
pattern_names = ['rotation', 'reflection', 'scaling', 'translation', 'color_mapping',
                 'symmetry', 'object_movement', 'counting', 'logical', 'composite']

for model_name, results in training_results.items():
    # Training history
    fig_history = create_training_plots(model_name, results['history'])
    fig_history.show()
    fig_history.write_html(f'{model_name}_training_history.html')
    
    # Confusion matrix
    fig_cm = create_confusion_matrix_plot(model_name, results['val_labels'], 
                                          results['val_preds'], pattern_names)
    fig_cm.show()
    fig_cm.write_html(f'{model_name}_confusion_matrix.html')

# Overall comparison
fig_comparison = create_performance_comparison()
fig_comparison.show()
fig_comparison.write_html('model_comparison.html')

## 6. Generate Detailed Metrics

In [ ]:
# Generate detailed metrics for each model
detailed_metrics = {}

for model_name, results in training_results.items():
    print(f"\n{'='*60}")
    print(f"{model_name.upper()} - Detailed Performance Report")
    print(f"{'='*60}")
    
    # Classification report
    report = classification_report(results['val_labels'], results['val_preds'],
                                   target_names=pattern_names, output_dict=True)
    
    # Store metrics
    detailed_metrics[model_name] = {
        'classification_report': report,
        'best_accuracy': results['best_acc'],
        'parameters': sum(p.numel() for p in models[model_name].parameters()),
        'training_time': len(results['history']['train_loss']) * 2.5  # Approximate minutes per epoch
    }
    
    # Print summary
    print(f"Best Validation Accuracy: {results['best_acc']:.4f}")
    print(f"Model Parameters: {detailed_metrics[model_name]['parameters']:,}")
    print(f"\nPer-Pattern Performance:")
    
    for pattern in pattern_names:
        if pattern in report:
            print(f"  {pattern:20s} - Precision: {report[pattern]['precision']:.3f}, "
                  f"Recall: {report[pattern]['recall']:.3f}, "
                  f"F1: {report[pattern]['f1-score']:.3f}")

# Save metrics to JSON
with open('training_metrics.json', 'w') as f:
    json.dump(detailed_metrics, f, indent=2)

## 7. Convert to ONNX Format

In [ ]:
import onnx
import onnxruntime

def export_to_onnx(model, model_name, example_input, device):
    """Export PyTorch model to ONNX format"""
    model.eval()
    model = model.to(device)
    
    # Prepare inputs based on model
    onnx_path = f'{model_name}_model.onnx'
    
    if model_name in ['minerva', 'iris']:
        # Two input models
        example_output = example_input.clone()
        inputs = (example_input, example_output)
        input_names = ['input_grid', 'output_grid']
    elif model_name == 'chronos':
        # Sequence input - export with single grid
        inputs = example_input
        input_names = ['input_grid']
    else:
        # Single input models
        inputs = example_input
        input_names = ['input_grid']
    
    # Get output names from model
    with torch.no_grad():
        if model_name in ['minerva', 'iris']:
            sample_out = model(example_input, example_output)
        elif model_name == 'chronos':
            sample_out = model([example_input])
        else:
            sample_out = model(example_input)
    
    output_names = list(sample_out.keys())
    
    # Export
    print(f"Exporting {model_name} to ONNX...")
    torch.onnx.export(
        model,
        inputs,
        onnx_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=input_names,
        output_names=output_names,
        dynamic_axes={'input_grid': {0: 'batch_size'}}
    )
    
    # Verify
    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    print(f"✓ {model_name} exported successfully to {onnx_path}")
    
    return onnx_path

# Export all models
example_input = torch.randn(1, 10, 30, 30).to(device)
onnx_models = {}

for model_name, model in models.items():
    # Load best weights
    model.load_state_dict(torch.load(f'{model_name}_best.pt'))
    
    # Export
    onnx_path = export_to_onnx(model, model_name, example_input, device)
    onnx_models[model_name] = onnx_path

## 8. Save All Artifacts

In [ ]:
# Create output directory locally in Colab
output_dir = './ARC_Models_2025'
os.makedirs(output_dir, exist_ok=True)

# Create subdirectories
os.makedirs(f'{output_dir}/pytorch', exist_ok=True)
os.makedirs(f'{output_dir}/onnx', exist_ok=True)
os.makedirs(f'{output_dir}/visualizations', exist_ok=True)
os.makedirs(f'{output_dir}/metrics', exist_ok=True)

# Save PyTorch models
for model_name in models.keys():
    !cp {model_name}_best.pt {output_dir}/pytorch/{model_name}_model.pt

# Save ONNX models
!cp *.onnx {output_dir}/onnx/

# Save visualizations
!cp *_training_history.html {output_dir}/visualizations/
!cp *_confusion_matrix.html {output_dir}/visualizations/
!cp model_comparison.html {output_dir}/visualizations/

# Save metrics
!cp training_metrics.json {output_dir}/metrics/

# Create model cards
model_cards = {}
for model_name, metrics in detailed_metrics.items():
    model_cards[model_name] = {
        'name': model_name.upper(),
        'description': models[model_name].description,
        'architecture': models[model_name].__class__.__name__,
        'parameters': metrics['parameters'],
        'best_accuracy': metrics['best_accuracy'],
        'training_time_minutes': metrics['training_time'],
        'input_shape': [10, 30, 30],
        'framework': 'PyTorch 2.0',
        'created_date': datetime.now().strftime('%Y-%m-%d'),
        'author': 'Andrew Jewell Sr.',
        'organization': 'AutomataNexus, LLC'
    }

with open(f'{output_dir}/model_cards.json', 'w') as f:
    json.dump(model_cards, f, indent=2)

# Generate Hailo conversion script for your local machine
hailo_script = '''#!/bin/bash
# Convert ARC Prize 2025 ONNX models to HEF format for Hailo-8
# Author: Andrew Jewell Sr.
# Date: September 26, 2024

echo "============================================================"
echo "ARC Prize 2025 - ONNX to HEF Conversion for Hailo-8"
echo "============================================================"

# Models to convert
MODELS=("minerva" "atlas" "iris" "chronos" "prometheus")
BASE_DIR="/mnt/d/opt/ARCPrize2025"

# Activate Hailo virtual environment
echo "Activating Hailo environment..."
HAILO_VENV="/mnt/c/Users/Juelz/hailo_venv_py310"
if [ -d "$HAILO_VENV" ]; then
    source $HAILO_VENV/bin/activate
    echo "✓ Hailo environment activated"
else
    echo "✗ Hailo environment not found at $HAILO_VENV"
    exit 1
fi

# Check if hailo command is available
if ! command -v hailo &> /dev/null; then
    echo "✗ Hailo command not found in the environment"
    echo "Please ensure Hailo DFC is installed"
    exit 1
fi

echo ""
echo "Starting conversion of ${#MODELS[@]} models..."
echo ""

# Create output directory
mkdir -p "$BASE_DIR/hef"

# Convert each model
for model in "${MODELS[@]}"; do
    echo "----------------------------------------"
    echo "Converting $model..."
    echo "----------------------------------------"
    
    ONNX_FILE="$BASE_DIR/models/onnx/${model}_model.onnx"
    
    if [ ! -f "$ONNX_FILE" ]; then
        echo "✗ ONNX file not found: $ONNX_FILE"
        echo "  Skipping..."
        continue
    fi
    
    echo "Found ONNX file: $ONNX_FILE"
    
    # Step 1: Parse ONNX to HAR
    echo "Step 1/3: Parsing ONNX to HAR..."
    hailo parser onnx \\
        --hw-arch hailo8 \\
        --onnx-model "$ONNX_FILE" \\
        --output-har-path "$BASE_DIR/${model}.har" \\
        --start-node-names input_grid
    
    if [ ! -f "$BASE_DIR/${model}.har" ]; then
        echo "✗ Failed to create HAR file"
        continue
    fi
    
    # Step 2: Optimize with random calibration set
    echo "Step 2/3: Optimizing model..."
    hailo optimize \\
        --hw-arch hailo8 \\
        --har "$BASE_DIR/${model}.har" \\
        --use-random-calib-set \\
        --output-har-path "$BASE_DIR/${model}_optimized.har"
    
    if [ ! -f "$BASE_DIR/${model}_optimized.har" ]; then
        echo "✗ Failed to optimize model"
        continue
    fi
    
    # Step 3: Compile to HEF
    echo "Step 3/3: Compiling to HEF..."
    hailo compiler \\
        --hw-arch hailo8 \\
        --har "$BASE_DIR/${model}_optimized.har" \\
        --output-hef-path "$BASE_DIR/hef/${model}.hef"
    
    if [ -f "$BASE_DIR/hef/${model}.hef" ]; then
        echo "✓ Successfully compiled: $BASE_DIR/hef/${model}.hef"
        echo "  Size: $(du -h $BASE_DIR/hef/${model}.hef | cut -f1)"
        
        # Clean up intermediate files
        rm -f "$BASE_DIR/${model}.har" "$BASE_DIR/${model}_optimized.har"
        echo "  Cleaned up intermediate files"
    else
        echo "✗ Failed to compile $model to HEF"
    fi
    
    echo ""
done

echo "============================================================"
echo "Conversion Summary"
echo "============================================================"
echo "HEF files created:"
for model in "${MODELS[@]}"; do
    HEF_FILE="$BASE_DIR/hef/${model}.hef"
    if [ -f "$HEF_FILE" ]; then
        echo "  ✓ $model: $(du -h $HEF_FILE | cut -f1)"
    else
        echo "  ✗ $model: Not created"
    fi
done

echo ""
echo "Next steps:"
echo "1. Copy HEF files to Raspberry Pi 5:"
echo "   scp $BASE_DIR/hef/*.hef Automata@192.168.0.54:/home/Automata/mydata/neural-nexus/arc2025/"
echo ""
echo "2. Test on Raspberry Pi with:"
echo "   hailortcli run /home/Automata/mydata/neural-nexus/arc2025/minerva.hef"
echo ""

# Deactivate virtual environment
deactivate 2>/dev/null || true
'''

with open(f'{output_dir}/convert_arc_to_hef.sh', 'w') as f:
    f.write(hailo_script)

print(f"\n✅ All artifacts saved to {output_dir}")
print("\nContents:")
print("  - PyTorch models (.pt)")
print("  - ONNX models (.onnx)")
print("  - Training visualizations (.html)")
print("  - Performance metrics (.json)")
print("  - Hailo conversion script (.sh)")
print("  - Model cards (.json)")

# Create a zip file with all outputs for easy download
!cd {output_dir} && zip -r ../ARC_Models_2025.zip *

print("\n📦 Created ARC_Models_2025.zip for download")

## 9. Summary Report

In [ ]:
print("\n" + "="*80)
print("ARC PRIZE 2025 - MODELS TRAINING COMPLETE")
print("="*80)
print(f"\nDate: {datetime.now().strftime('%B %d, %Y')}")
print(f"Author: Andrew Jewell Sr.")
print(f"Organization: AutomataNexus, LLC")
print("\n" + "-"*80)
print("MODEL PERFORMANCE SUMMARY")
print("-"*80)

for model_name in ['minerva', 'atlas', 'iris', 'chronos', 'prometheus']:
    if model_name in detailed_metrics:
        metrics = detailed_metrics[model_name]
        print(f"\n{model_name.upper()} - {models[model_name].description}")
        print(f"  Validation Accuracy: {metrics['best_accuracy']*100:.1f}%")
        print(f"  Parameters: {metrics['parameters']:,}")
        print(f"  Model Size: ~{metrics['parameters']*4/1024/1024:.1f} MB")
        print(f"  Training Time: {metrics['training_time']:.0f} minutes")

print("\n" + "-"*80)
print("NEXT STEPS")
print("-"*80)
print("1. Models saved to ./output/ARC_Models_2025")
print("2. Transfer ONNX models to Hailo device")
print("3. Run convert_to_hef.sh script on Hailo device")
print("4. Upload all formats to GitHub")
print("5. HTML reports with visualizations generated")
print("\n" + "="*80)

## 10. Download Results

In [ ]:
# Download the complete model package
from google.colab import files

print("Downloading ARC_Models_2025.zip...")
files.download('ARC_Models_2025.zip')

print("\n✅ Download complete!")
print("\nNext steps:")
print("1. Extract ARC_Models_2025.zip on your local machine")
print("2. Transfer ONNX models to your Raspberry Pi with Hailo-8")
print("3. Run the convert_to_hef.sh script on the Pi")
print("4. Push everything to GitHub")